# Canal bouillant - wall vaporisation

We simulate a 1D boiling channel with a heated wall to create vapor. We use a simple wall flux correlation diphasique_simple_lineaire to generate vapor. The idea is to check the flux repartition between the two phases is correctly computed by the polmac_P0 diffusion operator. (See canal.data for comments)

We test mainly the wall flux correlation diphasique_simple_lineaire (simple since used in TRUST). 

        flux_interfacial coef_constant { 
                                        liquide_eau 1e4 /* coeff echange liquide-interface (interface tjrs a Tsat) => petit echange => liquide loin de Tsat */
                                        gaz_eau 1e10 /* coeff echange gaz-interface (interface tjrs a Tsat) => grande echange => gaz toujours a Tsat */
                                        }
        flux_parietal diphasique_simple_lineaire { 
                                        coeff_echange_monophasique 5000 /* echange classique mono-phasique entre paroi et liquide */
                                        coeff_osv 5e-5 /* vapeur produite en paroi quand Tl = Ts-phi*coef_osv, donc si petit, ca fait de la vapeur quand liquide est a Tsat, si grand, production de vapeur tot */
                                        }

In [ ]:
from trustutils import run

run.introduction("Yannick Gorsse & Elie Saikali")

In [ ]:
import os

run.reset()
run.initBuildDirectory()

dis = [ ("PolyMAC_P0", "Polyedriser dom", "option_polymac { }"),
       ("VDF", "", "option_vdf { all_options }") ]

for d, dd, ddd in dis:
    run.addCaseFromTemplate("canal.data", f"{d}", {"dis" : d , "poly" : dd, "opts" : ddd})

run.printCases()
run.runCases()
run.tablePerf()

In [ ]:
from trustutils import plot

a=plot.Graph("Residus")
for d, _, _ in dis:
    a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/canal.dt_ev",label=f"{d}")

a.scale(yscale="log")

In [ ]:
for name, c in [
    ("TEMPERATURE_L", 0),
    ("TEMPERATURE_G", 0),
    ("PRESSION", 0),
    ("VITESSE_L", 2),
    ("VITESSE_G", 2),
    ("ALPHA", 0),
]:
    a = plot.Graph(name)
    for d, _, _ in dis:
        a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/canal_{name}_TRUST.son", compo=c, label=f"{d}")